# 🎯 บทที่ 2.5: Train/Test Split - การแบ่งข้อมูล

## 🎓 เรื่องราวของการสอบ

ลองนึกภาพคุณกำลังเตรียมสอบ:

### 😱 วิธีผิด (Data Leakage):
- ครูให้ดู **ข้อสอบจริง** ก่อนสอบ
- คุณท่องจำคำตอบทั้งหมด
- วันสอบ: ได้ 100 คะแนน! 🎉
- แต่... **ไม่ได้เรียนรู้อะไรจริงๆ**

### ✅ วิธีถูก (Proper Split):
- ครูให้ **แบบฝึกหัด** (Training Data)
- คุณฝึกทำ เข้าใจหลักการ
- วันสอบ: ข้อสอบ **ใหม่** ที่ไม่เคยเห็น (Test Data)
- ได้ 85 คะแนน → แสดงว่า**เรียนรู้จริง**! ✨

**Machine Learning ก็เหมือนกัน:**
- **Train Data** = แบบฝึกหัด (ให้โมเดลเรียนรู้)
- **Test Data** = ข้อสอบจริง (ใช้วัดความสามารถจริง)
- **Data Leakage** = ข้อมูล test รั่วไปใน train → โมเดล**โกง**!

In [ ]:
# Import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from collections import Counter
import warnings
warnings.filterwarnings('ignore')

plt.style.use('seaborn-v0_8-whitegrid')
print("✅ Libraries loaded!")

---

## 📊 Part 1: แบ่งข้อมูล CSV

### สัดส่วนมาตรฐาน:
- **70% Training** - เรียนรู้
- **15% Validation** - Fine-tune
- **15% Test** - ประเมินผลจริง

In [ ]:
# สร้างข้อมูลตัวอย่าง
np.random.seed(42)

n_samples = 1000
data = {
    'Feature1': np.random.randn(n_samples),
    'Feature2': np.random.randn(n_samples),
    'Feature3': np.random.randn(n_samples),
    'Target': np.random.choice([0, 1], n_samples)
}

df = pd.DataFrame(data)

print("📊 Dataset Overview:")
print(f"   Total samples: {len(df)}")
print(f"   Features: {df.columns.tolist()[:-1]}")
print(f"   Target: {df['Target'].unique()}")
print("\nFirst 5 rows:")
df.head()

In [ ]:
# วิธีที่ 1: แบ่ง 80-20 (Train-Test)
X = df.drop('Target', axis=1)
y = df['Target']

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,      # 20% test
    random_state=42     # ทำให้ผลเหมือนเดิมทุกครั้ง
)

print("📦 80-20 Split:")
print(f"   Training: {len(X_train)} samples ({len(X_train)/len(df)*100:.1f}%)")
print(f"   Test: {len(X_test)} samples ({len(X_test)/len(df)*100:.1f}%)")

In [ ]:
# วิธีที่ 2: แบ่ง 70-15-15 (Train-Val-Test)
# Step 1: แบ่ง 70-30
X_train, X_temp, y_train, y_temp = train_test_split(
    X, y,
    test_size=0.3,
    random_state=42
)

# Step 2: แบ่ง 30 เป็น 15-15
X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp,
    test_size=0.5,
    random_state=42
)

print("📦 70-15-15 Split:")
print(f"   Training: {len(X_train)} samples ({len(X_train)/len(df)*100:.1f}%)")
print(f"   Validation: {len(X_val)} samples ({len(X_val)/len(df)*100:.1f}%)")
print(f"   Test: {len(X_test)} samples ({len(X_test)/len(df)*100:.1f}%)")

# Visualization
sizes = [len(X_train), len(X_val), len(X_test)]
labels = ['Training\n70%', 'Validation\n15%', 'Test\n15%']
colors = ['#66b3ff', '#99ff99', '#ffcc99']
explode = (0.05, 0.05, 0.05)

plt.figure(figsize=(8, 8))
plt.pie(sizes, labels=labels, colors=colors, autopct='%1.0f%%',
        startangle=90, explode=explode, textprops={'fontsize': 12, 'weight': 'bold'})
plt.title('Train-Validation-Test Split', fontsize=14, fontweight='bold', pad=20)
plt.show()

---

## 🖼️ Part 2: แบ่งข้อมูลภาพ (Images)

### ความแตกต่างจาก CSV:
- ภาพมักจัดเก็บใน **โฟลเดอร์** แยกตาม class
- ต้องแบ่งโดยรักษา **สัดส่วนของแต่ละ class**

In [ ]:
# จำลองข้อมูลภาพ (file paths)
np.random.seed(42)

# สร้าง mock image paths
n_cats = 300
n_dogs = 200

cat_images = [f'images/cats/cat_{i}.jpg' for i in range(n_cats)]
dog_images = [f'images/dogs/dog_{i}.jpg' for i in range(n_dogs)]

all_images = cat_images + dog_images
all_labels = [0] * n_cats + [1] * n_dogs  # 0=cat, 1=dog

print("📸 Image Dataset:")
print(f"   Total images: {len(all_images)}")
print(f"   Cats: {n_cats}")
print(f"   Dogs: {n_dogs}")
print(f"   Ratio: {n_cats}:{n_dogs}")

In [ ]:
# แบ่งข้อมูลภาพ - ต้องใช้ stratify!
train_imgs, test_imgs, train_labels, test_labels = train_test_split(
    all_images, all_labels,
    test_size=0.2,
    stratify=all_labels,  # 🔑 สำคัญ! รักษาสัดส่วน class
    random_state=42
)

print("📦 Image Split (with stratify):")
print(f"\nTraining set:")
print(f"   Total: {len(train_imgs)}")
print(f"   Cats: {train_labels.count(0)} ({train_labels.count(0)/len(train_labels)*100:.1f}%)")
print(f"   Dogs: {train_labels.count(1)} ({train_labels.count(1)/len(train_labels)*100:.1f}%)")

print(f"\nTest set:")
print(f"   Total: {len(test_imgs)}")
print(f"   Cats: {test_labels.count(0)} ({test_labels.count(0)/len(test_labels)*100:.1f}%)")
print(f"   Dogs: {test_labels.count(1)} ({test_labels.count(1)/len(test_labels)*100:.1f}%)")

print("\n✅ สัดส่วน 60:40 ถูกรักษาไว้ทั้ง train และ test!")

---

## 🚨 Part 3: Data Leakage - อันตรายร้ายแรง!

### Data Leakage คืออะไร?
ข้อมูลจาก **test set รั่วเข้าไปใน training process**

### ผลที่ตามมา:
- Training accuracy สูงมาก 99%+
- Test accuracy แย่มาก 50-60%
- โมเดล**โกง** แต่ไม่รู้ตัว!

In [ ]:
# ตัวอย่าง Data Leakage แบบต่างๆ
print("🚨 Common Data Leakage Scenarios:\n")
print("="*70)

print("\n1️⃣ Scaling ก่อนแบ่ง (ผิด!)")
print("""
❌ WRONG:
   scaler.fit(all_data)          # ใช้ทั้ง train + test
   scaled_data = scaler.transform(all_data)
   train, test = split(scaled_data)

   → Test data รั่วเข้า training (ผ่าน mean, std)
""")

print("""
✅ CORRECT:
   train, test = split(data)     # แบ่งก่อน!
   scaler.fit(train)              # Fit เฉพาะ train
   train = scaler.transform(train)
   test = scaler.transform(test)  # ใช้ scaler เดียวกัน
""")

print("\n2️⃣ Feature Selection ก่อนแบ่ง (ผิด!)")
print("""
❌ WRONG:
   best_features = select_features(all_data)  # เห็น test!
   train, test = split(data[best_features])

✅ CORRECT:
   train, test = split(data)
   best_features = select_features(train)  # เห็นแค่ train
""")

print("\n3️⃣ Duplicate Data (ผิด!)")
print("""
❌ WRONG:
   data มีข้อมูลซ้ำ
   → แบ่ง train/test แบบสุ่ม
   → ข้อมูลเดียวกันอยู่ทั้ง train และ test!

✅ CORRECT:
   ลบ duplicates ก่อน
   แล้วค่อยแบ่ง
""")

In [ ]:
# Demo: Data Leakage vs No Leakage
from sklearn.datasets import make_classification
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# สร้างข้อมูล
X, y = make_classification(n_samples=1000, n_features=20, random_state=42)

print("🔬 Experiment: Data Leakage Impact\n")
print("="*70)

# ❌ วิธีผิด: Scale ก่อนแบ่ง
print("\n❌ WITH Data Leakage:")
scaler_wrong = StandardScaler()
X_scaled_wrong = scaler_wrong.fit_transform(X)  # Fit ทั้งหมด!
X_train_wrong, X_test_wrong, y_train_wrong, y_test_wrong = train_test_split(
    X_scaled_wrong, y, test_size=0.2, random_state=42
)

model_wrong = LogisticRegression(max_iter=1000)
model_wrong.fit(X_train_wrong, y_train_wrong)

train_acc_wrong = model_wrong.score(X_train_wrong, y_train_wrong)
test_acc_wrong = model_wrong.score(X_test_wrong, y_test_wrong)

print(f"   Train Accuracy: {train_acc_wrong:.4f}")
print(f"   Test Accuracy: {test_acc_wrong:.4f}")
print(f"   Gap: {abs(train_acc_wrong - test_acc_wrong):.4f}")

# ✅ วิธีถูก: แบ่งก่อน แล้วค่อย Scale
print("\n✅ WITHOUT Data Leakage (Correct):")
X_train_correct, X_test_correct, y_train_correct, y_test_correct = train_test_split(
    X, y, test_size=0.2, random_state=42
)

scaler_correct = StandardScaler()
X_train_correct = scaler_correct.fit_transform(X_train_correct)  # Fit เฉพาะ train!
X_test_correct = scaler_correct.transform(X_test_correct)        # Transform test

model_correct = LogisticRegression(max_iter=1000)
model_correct.fit(X_train_correct, y_train_correct)

train_acc_correct = model_correct.score(X_train_correct, y_train_correct)
test_acc_correct = model_correct.score(X_test_correct, y_test_correct)

print(f"   Train Accuracy: {train_acc_correct:.4f}")
print(f"   Test Accuracy: {test_acc_correct:.4f}")
print(f"   Gap: {abs(train_acc_correct - test_acc_correct):.4f}")

print("\n💡 สังเกต: Gap ที่เล็กกว่า = โมเดล generalize ได้ดีกว่า!")

---

## ⚖️ Part 4: Class Imbalance - ข้อมูลไม่สมดุล

### ปัญหา:
- Class หนึ่งมีข้อมูลเยอะกว่าอีก class มาก
- เช่น: มะเร็ง (1%) vs ปกติ (99%)

### ผลกระทบ:
- โมเดลทาย "ปกติ" ทุกครั้ง → Accuracy 99%!
- แต่**ไม่มีประโยชน์** (ไม่ได้จับมะเร็งเลย)

In [ ]:
# สร้างข้อมูลที่ Imbalance
np.random.seed(42)

# 90% Class 0, 10% Class 1
n_class0 = 900
n_class1 = 100

X_imb = np.vstack([
    np.random.randn(n_class0, 2),
    np.random.randn(n_class1, 2) + 2
])

y_imb = np.array([0] * n_class0 + [1] * n_class1)

# นับจำนวน
class_counts = Counter(y_imb)

print("⚖️ Imbalanced Dataset:")
print(f"   Total samples: {len(y_imb)}")
print(f"   Class 0 (Majority): {class_counts[0]} ({class_counts[0]/len(y_imb)*100:.1f}%)")
print(f"   Class 1 (Minority): {class_counts[1]} ({class_counts[1]/len(y_imb)*100:.1f}%)")
print(f"   Imbalance Ratio: {class_counts[0]/class_counts[1]:.1f}:1")

# Visualization
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Bar chart
axes[0].bar(['Class 0\n(Negative)', 'Class 1\n(Positive)'], 
            [class_counts[0], class_counts[1]], 
            color=['skyblue', 'coral'])
axes[0].set_ylabel('Count')
axes[0].set_title('Class Distribution (Imbalanced)', fontsize=13, fontweight='bold')
axes[0].grid(axis='y', alpha=0.3)

# Scatter plot
axes[1].scatter(X_imb[y_imb==0, 0], X_imb[y_imb==0, 1], 
               alpha=0.5, s=50, label=f'Class 0 (n={class_counts[0]})', c='skyblue')
axes[1].scatter(X_imb[y_imb==1, 0], X_imb[y_imb==1, 1], 
               alpha=0.7, s=50, label=f'Class 1 (n={class_counts[1]})', c='coral')
axes[1].set_xlabel('Feature 1')
axes[1].set_ylabel('Feature 2')
axes[1].set_title('Data Visualization', fontsize=13, fontweight='bold')
axes[1].legend()
axes[1].grid(alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# วิธีแก้ Imbalance
print("🛠️ Solutions for Class Imbalance:\n")
print("="*70)

print("\n1️⃣ Stratified Split (รักษาสัดส่วน):")
X_train_strat, X_test_strat, y_train_strat, y_test_strat = train_test_split(
    X_imb, y_imb,
    test_size=0.2,
    stratify=y_imb,  # 🔑 สำคัญ!
    random_state=42
)

train_counts = Counter(y_train_strat)
test_counts = Counter(y_test_strat)

print(f"   Training set:")
print(f"      Class 0: {train_counts[0]} ({train_counts[0]/len(y_train_strat)*100:.1f}%)")
print(f"      Class 1: {train_counts[1]} ({train_counts[1]/len(y_train_strat)*100:.1f}%)")

print(f"\n   Test set:")
print(f"      Class 0: {test_counts[0]} ({test_counts[0]/len(y_test_strat)*100:.1f}%)")
print(f"      Class 1: {test_counts[1]} ({test_counts[1]/len(y_test_strat)*100:.1f}%)")

print("\n   ✅ สัดส่วน 90:10 ถูกรักษาทั้ง train และ test!")

In [ ]:
print("\n2️⃣ Class Weights (ให้ความสำคัญกับ minority class):")
print("""
from sklearn.utils.class_weight import compute_class_weight

class_weights = compute_class_weight(
    'balanced',
    classes=np.unique(y_train),
    y=y_train
)

model.fit(X_train, y_train, class_weight=dict(enumerate(class_weights)))
""")

print("\n3️⃣ SMOTE (Synthetic Minority Over-sampling):")
print("""
from imblearn.over_sampling import SMOTE

smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_train, y_train)

# สร้างข้อมูล synthetic ให้ minority class
""")

print("\n4️⃣ Undersampling (ลด majority class):")
print("""
from imblearn.under_sampling import RandomUnderSampler

rus = RandomUnderSampler(random_state=42)
X_resampled, y_resampled = rus.fit_resample(X_train, y_train)

# ลดจำนวน majority class ให้เท่า minority
""")

print("\n5️⃣ Use Better Metrics (อย่าดูแค่ Accuracy!):")
print("""
from sklearn.metrics import classification_report, f1_score, precision_recall_curve

# ใช้ metrics เหล่านี้:
- Precision: ทายบวกแล้วถูกกี่%
- Recall: ของจริงบวกจับได้กี่%
- F1-Score: ค่ากลาง Precision และ Recall
- ROC-AUC: วัดความสามารถแยก class
""")

---

## 🎓 Part 5: Cross-Validation

### ปัญหาของ Single Split:
- ผลลัพธ์ขึ้นอยู่กับว่า **แบ่งยังไง**
- อาจโชคดี/โชคร้าย

### วิธีแก้: K-Fold Cross-Validation
- แบ่งเป็น K ส่วน
- เทรน K ครั้ง (แต่ละครั้งใช้ส่วนต่างกันเป็น test)
- เฉลี่ยผลลัพธ์

In [ ]:
from sklearn.model_selection import cross_val_score, KFold

# 5-Fold Cross-Validation
kfold = KFold(n_splits=5, shuffle=True, random_state=42)

print("🔄 5-Fold Cross-Validation:\n")
print("="*70)

# Visualize folds
fig, axes = plt.subplots(5, 1, figsize=(12, 8))

for fold, (train_idx, test_idx) in enumerate(kfold.split(X)):
    train_mask = np.zeros(len(X))
    test_mask = np.zeros(len(X))
    train_mask[train_idx] = 1
    test_mask[test_idx] = 1
    
    axes[fold].barh([0], [len(train_idx)], left=[0], height=0.5, color='skyblue', label='Train')
    axes[fold].barh([0], [len(test_idx)], left=[len(train_idx)], height=0.5, color='coral', label='Test')
    axes[fold].set_xlim([0, len(X)])
    axes[fold].set_yticks([])
    axes[fold].set_title(f'Fold {fold+1}: Train={len(train_idx)}, Test={len(test_idx)}', 
                         fontsize=11, fontweight='bold')
    if fold == 0:
        axes[fold].legend(loc='upper right')

plt.tight_layout()
plt.show()

print("\n✅ แต่ละ fold ใช้ส่วนต่างกันเป็น test")
print("   → ได้ผลลัพธ์ที่เชื่อถือได้มากกว่า single split!")

---

## 📝 สรุปบทเรียน

### ✅ Key Takeaways:

#### 1. **Train/Test Split**
- CSV: `train_test_split()`
- Images: ใช้ `stratify` รักษาสัดส่วน class
- สัดส่วนแนะนำ: 70-15-15 หรือ 80-20

#### 2. **Data Leakage (อันตราย!)**
- ❌ **อย่า** scale/transform ก่อนแบ่ง
- ✅ **ต้อง** แบ่งก่อน แล้วค่อย fit scaler บน train เท่านั้น
- ลบ duplicates ก่อนแบ่ง

#### 3. **Class Imbalance**
- ใช้ `stratify=y` ตอนแบ่ง
- Class weights
- SMOTE / Undersampling
- ใช้ metrics ที่เหมาะสม (F1, Precision, Recall)

#### 4. **Cross-Validation**
- เชื่อถือได้กว่า single split
- แนะนำ 5-fold หรือ 10-fold
- ใช้เวลานานกว่า

### 🚀 Next Step:

👉 [Optimizers Comparison - เปรียบเทียบ Optimizers](06_optimizers_comparison.ipynb)

เรียนรู้ว่า optimizer แต่ละตัวทำงานยังไง และเลือกใช้อย่างไร!